In [1]:
import cv2
import pandas as pd
from imutils.video import FPS

In [2]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ultralytics import YOLO

In [4]:
import math
class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [5]:
model=YOLO('yolov8s.pt')
# model=YOLO('yolov8m.pt')

In [6]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('veh2.mp4')


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0

tracker=Tracker()

cy1=323
cy2=367
offset=6

vh_down={}
counter=[]

vh_up={}
counter1=[]


while True:    
    ret,frame = cap.read()
    fps = FPS().start() 
    
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
    fps.update()
    fps.stop()
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        
        if cy1<(cy+offset) and cy1>(cy-offset):
            vh_down[id]=cy
        if id in vh_down:
            if cy2<(cy+offset) and cy2>(cy-offset):
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter.count(id)==0:
                    counter.append(id)
        ##Going down
        if cy2<(cy+offset) and cy2>(cy-offset):
            vh_up[id]=cy
        if id in vh_up:
            if cy1<(cy+offset) and cy1>(cy-offset):
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter1.count(id)==0:
                    counter1.append(id)

    cv2.line(frame,(259,cy1),(811,cy1),(255,255,255),1)
    cv2.putText(frame,('1line'),(274,318),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    cv2.line(frame,(154,cy2),(913,cy2),(255,255,255),1)
    cv2.putText(frame,('2line'),(154,365),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    d=(len(counter))
    cv2.putText(frame,('goingdown: -')+str(d),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    u=(len(counter1))
    cv2.putText(frame,('goingup: -')+str(u),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    txt_fps = "FPS: {:.2f}".format(fps.fps())
    cv2.putText(frame, txt_fps,(60,180),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    print(counter)
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()




0: 320x640 1 person, 3 cars, 1 fire hydrant, 2655.0ms
Speed: 24.9ms preprocess, 2655.0ms inference, 32.9ms postprocess per image at shape (1, 3, 320, 640)
[]

0: 320x640 1 person, 3 cars, 1 fire hydrant, 1046.8ms
Speed: 20.9ms preprocess, 1046.8ms inference, 15.0ms postprocess per image at shape (1, 3, 320, 640)
[]

0: 320x640 1 person, 3 cars, 1 fire hydrant, 1198.3ms
Speed: 28.9ms preprocess, 1198.3ms inference, 4.0ms postprocess per image at shape (1, 3, 320, 640)
[]

0: 320x640 1 person, 3 cars, 1 fire hydrant, 1698.1ms
Speed: 15.0ms preprocess, 1698.1ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)
[]

0: 320x640 1 person, 3 cars, 1 fire hydrant, 699.6ms
Speed: 119.2ms preprocess, 699.6ms inference, 8.0ms postprocess per image at shape (1, 3, 320, 640)
[]
[359, 350]

0: 320x640 3 cars, 1 fire hydrant, 731.6ms
Speed: 16.0ms preprocess, 731.6ms inference, 7.0ms postprocess per image at shape (1, 3, 320, 640)
[]
[250, 12]
[250, 10]

0: 320x640 2 persons, 3 cars, 1